In [2]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import os
from models.binarized_modules import binarized
import math
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm import tqdm
# from binarized_modules import  BinarizeLinear,BinarizeConv2d

In [3]:
import sys
sys.path.append('/shares/bulk/earapidis/dev/Fast-Crossbar-Sim/python')
# sys.path.append('/home/earapidis/Fast-Crossbar-Sim/python')
from crossbar import VectorSim, ParallelSim, _task
from tqdm import tqdm

In [4]:
os.getcwd()

'/shares/bulk/earapidis/dev/BinarizedNN'

In [5]:
cuda = False

In [6]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

In [7]:
test_batch_size=2

In [8]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=False, **kwargs)

In [9]:
# from mnist_bnn import Net
from models.lenet_5 import BinarizedLeNet5_BN as Net

model = Net()
if cuda:
    torch.cuda.set_device(0)
    model.cuda()


In [10]:
# model_path = os.path.join(models_path,f"epoch_7.pth")
model_idx = 3
models_path = os.path.abspath(f"/shares/bulk/earapidis/dev/BinarizedNN/saved_models/lenet_5/model_{model_idx}")
# models_path = os.path.abspath(f"/home/earapidis/BinarizedNN/saved_models/lenet_5/model_{model_idx}")
model_path = os.path.join(models_path,f"epoch_15.pth")
# model_path = os.path.join(models_path,f"best.pth")
model = Net()
model.load_state_dict(torch.load(model_path))
if cuda:
    torch.cuda.set_device(0)
    model.cuda()

In [11]:
model.eval()

BinarizedLeNet5_BN(
  (conv1): BinarizeConv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh1): Hardtanh(min_val=-1.0, max_val=1.0)
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): BinarizeConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh2): Hardtanh(min_val=-1.0, max_val=1.0)
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (fc1): BinarizeLinear(in_features=256, out_features=120, bias=True)
  (bn_fc1): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh3): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc2): BinarizeLinear(in_features=120, out_features=84, bias=True)
  (bn_fc2): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (htanh4): Hardtanh(min_val=-1.0, max_val=1.0)
  (fc3): BinarizeLinear(in_features=84

In [12]:
images, labels = next(iter(test_loader))
print(f"image shape: {images.shape}")

image shape: torch.Size([2, 1, 28, 28])


In [13]:

x=model.conv1(images)
x=model.bn1(x)
x=model.htanh1(x)
x=model.pool1(x)
x = x.detach()

In [14]:
inputs = images
# inputs = x
inputs = binarized(inputs)
inputs.shape

torch.Size([2, 1, 28, 28])

In [15]:
filters = model.conv1.weight.data
filters_b = binarized(filters)
filters_b.shape

torch.Size([6, 1, 5, 5])

In [16]:
# COUT = 50
# CIN = 6
# Kh = 5
# Kw = 5
# filters = torch.randn(COUT,CIN,Kh,Kw)
# filters = binarized(filters)

COUT, CIN, Kh, Kw = filters.shape
N, CIN, Hi,Wi = inputs.shape
# Hi = 24
# Wi = 24
# N = 1
padding = 0 
Num_rows = 32
Num_Columns = 32                    
Hout = Hi + 2*padding - Kh + 1
Wout = Wi + 2*padding - Kw + 1


# filters = torch.randn(COUT, CIN, Kh, Kw)


In [20]:
def checkerboard_last_cols(arr: torch.Tensor, C: int) -> torch.Tensor:
    n, m = arr.shape
    if C == 0:
        return arr.clone()  # return a copy even if unchanged
    if C > m:
        raise ValueError("C cannot be larger than the number of columns m.")
    
    arr_copy = arr.clone()
    rows = torch.arange(n).view(-1, 1)                
    cols = torch.arange(m - C, m).view(1, -1)         

    pattern = (rows + cols) % 2
    arr_copy[:, -C:] = pattern
    return arr_copy
# Example
A = torch.ones((6, 8), dtype=torch.int)
A=checkerboard_last_cols(A, C=2)
print(A)


tensor([[1, 1, 1, 1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1, 1, 1, 0]], dtype=torch.int32)


In [119]:
def compliment(x):
    x = x.clone()
    neg = -1*x
    pos = x

    pos[pos==-1] = 0
    neg[neg==-1] = 0
    return pos, neg

In [120]:
pos_inputs, neg_inputs = compliment(inputs)
pos_filters, neg_filters = compliment(filters_b)

print(pos_filters.shape)
print(pos_inputs.shape)

torch.Size([6, 1, 5, 5])
torch.Size([2, 1, 28, 28])


In [121]:
new_weights = torch.empty((COUT,CIN,2*(Kh*Kw)))

pos = pos_filters.reshape(pos_filters.shape[0],pos_filters.shape[1],-1)
neg = neg_filters.reshape(neg_filters.shape[0],neg_filters.shape[1],-1)
kernel_size = Kh*Kw

for i in range(kernel_size):

    new_weights[:,:,2*i] = pos[:,:,i]
    new_weights[:,:,2*i+1] = neg[:,:,i]
new_weights.shape

torch.Size([6, 1, 50])

In [122]:
def map_conv2d(x,w,padding=0):
    _COUT_, _CIN_, _Kh_, _Kw_ = w.shape
    _N_, _CIN_, _Hi_, _Wi_ = x.shape
    pos_w, neg_w = compliment(w)
    mapped_w = torch.empty((_COUT_,_CIN_,2*(_Kh_*_Kw_)))

    pos_w = pos_w.reshape(_COUT_,_CIN_,-1)
    neg_w = neg_w.reshape(_COUT_,_CIN_,-1)
    kernel_size = _Kh_*_Kw_
    for i in range(kernel_size):

        mapped_w[:,:,2*i] = pos_w[:,:,i]
        mapped_w[:,:,2*i+1] = neg_w[:,:,i]
    
    _Hout_ = _Hi_ + 2*padding - _Kh_ + 1
    _Wout_ = _Wi_ + 2*padding - _Kw_ + 1
    
    mapped_x = torch.empty((_N_,_Hout_,_Wout_,_CIN_,2*kernel_size))
    pos_x, neg_x = compliment(x)
    
    for ii in range(_Hout_):
        for jj in range(_Wout_):
            _pos_ = pos_x[:,:,ii:ii+_Kh_,jj:jj+_Kw_]
            _neg_ = neg_x[:,:,ii:ii+_Kh_,jj:jj+_Kw_]
            # print(_pos_.shape)
            _pos_ = _pos_.reshape(_N_,_CIN_,-1)
            _neg_ = _neg_.reshape(_N_,_CIN_,-1)
            one_kernel = torch.empty(_N_,_CIN_,2*kernel_size)
            for z in range(kernel_size):
                one_kernel[:,:,2*z]=_pos_[:,:,z]
                one_kernel[:,:,2*z+1]=_neg_[:,:,z]
            mapped_x[:,ii,jj,:,:] = one_kernel
    return mapped_x,mapped_w

In [123]:
new_weights, new_inputs = map_conv2d(inputs,filters_b)

In [124]:
def get_conv2d_output(I,Kh, Kw, CIN):
    out = 2*I - Kh*Kw*CIN
    return out

In [125]:
ref = F.conv2d(inputs, filters_b, padding=0)
# ref

In [126]:
# out_2d = torch.empty((N,COUT,Hout,Wout))

# for ii in range(Hout):
#     for jj in range(Wout):
#         one_pixel = new_inputs[:,ii,jj,:,:]
#         # print(one_pixel.shape)
#         out_1d = F.conv1d(one_pixel,new_weights)
#         out_1d = out_1d.squeeze(-1)
#         # print(out_1d.shape)
#         out_2d[:,:,ii,jj] = out_1d
# out_2d = get_conv2d_output(out_2d,Kh,Kw,CIN)
# torch.equal(out_2d,ref)

In [127]:
# def torch_conv_2d(crossbar_inputs,crossbar_weights,_COUT_,Total_dim):
#     _N_,_HOUT_,_WOUT_,crossbar_y,Num_rows = crossbar_inputs.shape
#     _,crossbar_x,_,Num_Columns = crossbar_weights.shape
#     output_conv_2d = torch.zeros(_N_,_COUT_,_HOUT_,_WOUT_)

#     columns_per_crossbar = math.floor(_COUT_/crossbar_x)

#     for ii in range(_HOUT_):
#         for jj in range(_WOUT_):
#             mac_out_columns = torch.zeros((_N_,_COUT_))
#             for cy in range(crossbar_y):
#                 for cx in range(crossbar_x):
#                     tmp_x=crossbar_inputs[:,ii,jj,cy,:]
#                     tmp_w=crossbar_weights[cy,cx,:,:]
#                     checkerboard_last_cols(tmp_w,Num_Columns-columns_per_crossbar)
#                     # tmp_w=crossbar_weights
#                     # print(tmp_x.shape)
#                     # print(tmp_w.shape)
#                     # _out_ = torch.matmul(tmp_w,tmp_x)
#                     _out_ = torch.matmul(tmp_x,tmp_w)

#                     column_start_idx = cx*columns_per_crossbar
#                     if cx==crossbar_x-1:
#                         column_end_idx=Total_dim
#                     else:
#                         column_end_idx = (cx+1)*columns_per_crossbar
                    
#                     mac_out_columns[:,column_start_idx:column_end_idx] += _out_[:,:columns_per_crossbar]
#             output_conv_2d[:,:,ii,jj]=mac_out_columns
#     return output_conv_2d


In [ ]:

def run_tile(ii,jj,cy, cx, tmp_x_np, tmp_w_np, Num_rows, Num_Columns, columns_per_crossbar, Total_dim, mode,transient):
    tmp_w_np= checkerboard_last_cols(tmp_w_np, tmp_w_np.shape[1] - columns_per_crossbar)
    _, _out_np = _task(((ii,jj), tmp_x_np), tmp_w_np, Num_rows, Num_Columns, mode, transient)

    column_start_idx = cx * columns_per_crossbar
    column_end_idx = (cx + 1) * columns_per_crossbar
    # column_end_idx = Total_dim if tmp_w_np.shape[1] - columns_per_crossbar < columns_per_crossbar else (cx + 1) * columns_per_crossbar
    return ((ii, jj, column_start_idx, column_end_idx), _out_np[:, :columns_per_crossbar])

def cim_conv_2d(crossbar_inputs, crossbar_weights, _COUT_, mode, Total_dim, transient,max_workers=None):
    _N_, _HOUT_, _WOUT_, crossbar_y, Num_rows = crossbar_inputs.shape
    _, crossbar_x, _, Num_Columns = crossbar_weights.shape
    output_conv_2d = torch.zeros(_N_, _COUT_, _HOUT_, _WOUT_)

    columns_per_crossbar = math.floor(_COUT_ / crossbar_x)

    # Preconvert weights to NumPy once (this is reused for each (ii, jj))
    weights_np = [[crossbar_weights[cy, cx].numpy() for cx in range(crossbar_x)] for cy in range(crossbar_y)]

    tasks = []
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        for ii in range(_HOUT_):
            for jj in range(_WOUT_):

                for cy in range(crossbar_y):
                    for cx in range(crossbar_x):
                        tmp_x = crossbar_inputs[:, ii, jj, cy, :].numpy()
                        tmp_w = weights_np[cy][cx]
                        tasks.append((ii,jj,cy, cx, tmp_x, tmp_w, Num_rows, Num_Columns, columns_per_crossbar, Total_dim, mode, transient))

        futures = [executor.submit(run_tile, *t) for t in tasks]
        for f in tqdm(as_completed(futures),total=len(tasks)):
            (ii,jj, col_start, col_end), out_np = f.result()
            out = torch.from_numpy(out_np)
            output_conv_2d[:, col_start:col_end, ii, jj] += out

    return output_conv_2d


In [129]:
def _conv2d_tile_(x,w,Num_rows,Num_Columns,mode,max_workers,transient=False):
    _N_,_HOUT_, _WOUT_,_CIN_, _kernel_size_ = x.shape
    _COUT_, _CIN_, _kernel_size_ = w.shape
    output_conv_2d = torch.zeros(_N_,_COUT_,_HOUT_,_WOUT_)

    whole_input_size = _CIN_*_kernel_size_
    print(f"total inputs: {whole_input_size}")
    crossbar_y = math.ceil((_CIN_*_kernel_size_)/Num_rows)
    crossbar_x = math.ceil(_COUT_/Num_Columns)

    crossbar_weights = torch.zeros((crossbar_y,crossbar_x,Num_rows,Num_Columns))
    crossbar_inputs = torch.zeros((_N_,_HOUT_,_WOUT_,crossbar_y,Num_rows))

    rows_per_crossbar = math.floor(whole_input_size/crossbar_y)

    flatten_x = x.reshape(*x.shape[:-2],-1)
    print(flatten_x.shape)
    flatten_w = w.reshape(*w.shape[:-2],-1).T
    print(flatten_w.shape)
    print(crossbar_inputs.shape)
    print(f"weights shape: {crossbar_weights.shape}")
    print(f"inputs shape: {crossbar_inputs.shape}")

    columns_per_crossbar = math.floor(_COUT_/crossbar_x)

    for ii in range(crossbar_y):
        row_start_idx = ii*rows_per_crossbar
        # if ii==crossbar_y-1:
        #     row_end_idx = flatten_x.shape[-1]
        # else:
        row_end_idx = (ii+1)*rows_per_crossbar
        # print(start_idx,end_idx)
        crossbar_inputs[:,:,:,ii,:rows_per_crossbar] = flatten_x[:,:,:,row_start_idx:row_end_idx]
    # for ii in range(0,whole_input_size,step=inpu)

        for jj in range(crossbar_x):
            column_start_idx = jj*columns_per_crossbar
            # if jj==crossbar_x-1:
            #     column_end_idx=flatten_w.shape[-1]
            # else:
            column_end_idx = (jj+1)*columns_per_crossbar
            
            crossbar_weights[ii,jj,:rows_per_crossbar,:columns_per_crossbar] = flatten_w[row_start_idx:row_end_idx,column_start_idx:column_end_idx]


    # for n in range(_N_):
    output_conv_2d = cim_conv_2d(crossbar_inputs,crossbar_weights,_COUT_,mode,flatten_w.shape[-1],transient=transient,max_workers=max_workers)
    # output_conv_2d = torch_conv_2d(crossbar_inputs,crossbar_weights,_COUT_,flatten_w.shape[-1])
    return output_conv_2d


In [130]:
def conv2d_tile(x,w,Num_rows,Num_Columns,mode,max_workers,transient=False):
    _N_, _CIN_, _Hi_, _Wi_ = x.shape
    _COUT_, _CIN_, _Kh_, _Kw_ = w.shape
    mapped_x, mapped_w = map_conv2d(x,w)
    output_conv_2d = _conv2d_tile_(mapped_x,mapped_w,Num_rows,Num_Columns,mode,max_workers,transient)
    output_conv_2d = get_conv2d_output(output_conv_2d,_Kh_,_Kw_,_CIN_)
    
    return output_conv_2d


In [131]:
# output_conv_2d_cs=_conv2d_tile_()
# output_conv_2d_cs = conv2d_tile(inputs,filters_b,Num_rows,Num_Columns,mode="cs",max_workers=20,transient=False)
# torch.equal(output_conv_2d_cs,ref)

In [132]:
def con2d(x,w,Num_rows,Num_Columns,mode,max_workers,transient=False):
    padding = 0
    _N_, _CIN_, _Hi_, _Wi_ = x.shape
    _COUT_, _CIN_, _Kh_, _Kw_ = w.shape
    _HOUT_ = _Hi_ + 2*padding - _Kh_ + 1
    _WOUT_ = _Wi_ + 2*padding - _Kw_ + 1
    output_conv_2d = torch.zeros(_N_,_COUT_,_HOUT_,_WOUT_)
    for idx in range(_N_):
        tmp_x = x[idx].unsqueeze(0)
        output_conv_2d[idx,:,:,:] = conv2d_tile(tmp_x,w,Num_rows,Num_Columns,mode,max_workers,transient=False)
    return output_conv_2d


In [133]:
output_conv_2d_cs = con2d(inputs,filters_b,Num_rows,Num_Columns,mode="cs",max_workers=20,transient=False)
torch.equal(output_conv_2d_cs,ref)

total inputs: 50
torch.Size([1, 24, 24, 50])
torch.Size([50, 6])
torch.Size([1, 24, 24, 2, 32])
weights shape: torch.Size([2, 1, 32, 32])
inputs shape: torch.Size([1, 24, 24, 2, 32])


100%|██████████| 1152/1152 [00:03<00:00, 314.01it/s]


total inputs: 50
torch.Size([1, 24, 24, 50])
torch.Size([50, 6])
torch.Size([1, 24, 24, 2, 32])
weights shape: torch.Size([2, 1, 32, 32])
inputs shape: torch.Size([1, 24, 24, 2, 32])


100%|██████████| 1152/1152 [00:03<00:00, 310.56it/s]


True

In [134]:
output_conv_2d_gs=con2d(inputs,filters_b,Num_rows,Num_Columns,mode="gs",max_workers=20,transient=False)

total inputs: 50
torch.Size([1, 24, 24, 50])
torch.Size([50, 6])
torch.Size([1, 24, 24, 2, 32])
weights shape: torch.Size([2, 1, 32, 32])
inputs shape: torch.Size([1, 24, 24, 2, 32])


100%|██████████| 1152/1152 [00:03<00:00, 306.31it/s]


total inputs: 50
torch.Size([1, 24, 24, 50])
torch.Size([50, 6])
torch.Size([1, 24, 24, 2, 32])
weights shape: torch.Size([2, 1, 32, 32])
inputs shape: torch.Size([1, 24, 24, 2, 32])


100%|██████████| 1152/1152 [00:03<00:00, 320.26it/s]


In [135]:
ref_b = binarized(ref)

In [136]:
ref.shape

torch.Size([2, 6, 24, 24])

In [137]:
output_conv_2d_cs_b = binarized(output_conv_2d_cs)
output_conv_2d_gs_b = binarized(output_conv_2d_gs)
with np.printoptions(threshold=float('inf')):
    print(ref_b - output_conv_2d_gs_b)
    # print(ref_b - output_conv_2d_cs_b)

tensor([[[[-2., -2., -2.,  ..., -2., -2., -2.],
          [-2., -2., -2.,  ..., -2., -2., -2.],
          [-2., -2., -2.,  ..., -2., -2., -2.],
          ...,
          [-2., -2., -2.,  ..., -2., -2., -2.],
          [-2., -2., -2.,  ..., -2., -2., -2.],
          [-2., -2., -2.,  ..., -2., -2., -2.]],

         [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

         [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          ...,
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

         [[-2., -2., -2.,  ..., -2., -2., -2.],
          [-2., -2., -2.,  ..., -2., 

In [179]:
fc_weight = model.fc1.weight.data
fc_weight_b = binarized(fc_weight)
fc_weight_b.shape


torch.Size([120, 256])

In [201]:
fc_input = torch.randn(test_batch_size,fc_weight_b.shape[1])
fc_input_b = binarized(fc_input)
fc_input_b.shape

torch.Size([2, 256])

In [202]:
def map_fc(x,w):
    print(x.shape)
    print(w.shape)
    M, N = w.shape
    _N_, M = x.shape
    pos_w, neg_w = compliment(w) 
    pos_x, neg_x = compliment(x)

    mapped_w = torch.empty((2*M,N))
    mapped_x = torch.empty((_N_,2*M))

    for idx in range(M):
        _pos_w_ = pos_w[idx]
        _neg_w_ = neg_w[idx]
        mapped_w[2*idx]=_pos_w_
        mapped_w[2*idx+1]=_neg_w_

        _pos_x_ = pos_x[:,idx]
        _neg_x_ = neg_x[:,idx]

        mapped_x[:,2*idx] = _pos_x_
        mapped_x[:,2*idx+1] = _neg_x_

    return mapped_x,mapped_w

In [203]:
def get_fc_output(I, M):
    out = 2*I - M
    return out

In [207]:
new_fc_inputs, new_fc_weights = map_fc(fc_input_b,fc_weight_b.T)
out_2d = F.linear(new_fc_inputs,new_fc_weights.T)
out_2d = get_fc_output(out_2d,fc_input.shape[-1])
print(out_2d.shape)
ref_fc = F.linear(fc_input_b,fc_weight_b)

torch.equal(ref_fc,out_2d)

torch.Size([2, 256])
torch.Size([256, 120])
torch.Size([2, 120])


True

In [208]:
ref_fc.shape

torch.Size([2, 120])

In [ ]:
def  fc_linear(crossbar_inputs,crossbar_weights,N,mode,max_workers,transient):
    crossbar_y,crossbar_x,Num_rows,Num_Columns = crossbar_weights.shape
    _N_, crossbar_y, Num_rows = crossbar_inputs.shape
    output = torch.zeros(_N_,N)
    columns_per_crossbar = math.floor(N/crossbar_x)
    
    tasks = []

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        for ii in range(crossbar_y):
            for jj in range(crossbar_x):
                column_start_idx = jj*columns_per_crossbar
                column_end_idx = (jj+1)*columns_per_crossbar
                tmp_x = crossbar_inputs[:,ii]
                tmp_w = crossbar_weights[ii][jj]
                tmp_w=checkerboard_last_cols(tmp_w,Num_Columns-columns_per_crossbar)
                # out_matmul = torch.matmul(tmp_x,tmp_w)
                args = (((ii,jj),tmp_x),tmp_w,Num_rows,Num_Columns,mode,transient)
                tasks.append(args)
        futures = [executor.submit(_task, *t) for t in tasks]
        for f in tqdm(as_completed(futures),total=len(tasks)):
            (ii,jj), out_matmul = f.result()
            column_start_idx = jj*columns_per_crossbar
            column_end_idx = (jj+1)*columns_per_crossbar
            out_matmul = torch.from_numpy(out_matmul)
            output[:,column_start_idx:column_end_idx] += out_matmul[:,:columns_per_crossbar]

    return output

In [210]:
def _fc_tile_(x,w, Num_rows,Num_Columns,mode,max_workers,transient):
    _N_ , M = x.shape
    M , N = w.shape

    crossbar_y = math.ceil(M/Num_rows)
    crossbar_x = math.ceil(N/Num_Columns)
    # print("crossbar grid",crossbar_y,crossbar_x)

    crossbar_inputs = torch.zeros((_N_,crossbar_y,Num_rows))
    crossbar_weights = torch.zeros((crossbar_y,crossbar_x,Num_rows,Num_Columns))

    rows_per_crossbar = math.floor(M/crossbar_y)
    columns_per_crossbar = math.floor(N/crossbar_x)

    # print(rows_per_crossbar, columns_per_crossbar)

    for ii in range(crossbar_y):
        row_start_idx = ii*rows_per_crossbar
        # if ii==crossbar_y-1:
        #     row_end_idx = M
        # else:
        row_end_idx = (ii+1)*rows_per_crossbar
        crossbar_inputs[:,ii,:rows_per_crossbar] = x[:,row_start_idx:row_end_idx]
    # for ii in range(0,whole_input_size,step=inpu)
        # print(row_start_idx,row_end_idx)

        for jj in range(crossbar_x):
            column_start_idx = jj*columns_per_crossbar
            # if jj==crossbar_x-1:
            #     column_end_idx=N
            # else:
            column_end_idx = (jj+1)*columns_per_crossbar
            # print(column_start_idx,column_end_idx)
            
            crossbar_weights[ii,jj,:rows_per_crossbar,:columns_per_crossbar] = w[row_start_idx:row_end_idx,column_start_idx:column_end_idx]
        
    # print(f"crossbar weigths : {crossbar_weights.shape}")
    # print(f"crossbar inputs : {crossbar_inputs.shape}")
    return fc_linear(crossbar_inputs,crossbar_weights,N,mode,max_workers,transient)

In [211]:
def fc_tile(x,w, Num_rows,Num_Columns,mode,max_workers,transient):
    # w = w.T
    M , N = w.shape
    _N_ , M = x.shape
    mapped_x, mapped_w = map_fc(x,w)
    output_fc = _fc_tile_(mapped_x,mapped_w,Num_rows,Num_Columns,mode,max_workers,transient)
    output_fc = get_fc_output(output_fc,M)
    return output_fc

def fc(x,w, Num_rows,Num_Columns,mode,max_workers,transient):
    # w = w.T
    print(w.shape)
    M , N = w.shape
    _N_ , M = x.shape
    output_fc = torch.empty((_N_,N))

    for idx in range(_N_):
        tmp_x = x[idx].unsqueeze(0)
        output_fc[idx,:] = fc_tile(tmp_x,w, Num_rows,Num_Columns,mode,max_workers,transient)
    return output_fc

In [214]:
output_fc = fc(fc_input_b,fc_weight_b.T,Num_rows,Num_Columns,"cs",max_workers=8,transient=False)


torch.Size([256, 120])
torch.Size([1, 256])
torch.Size([256, 120])


100%|██████████| 64/64 [00:00<00:00, 97.10it/s] 


torch.Size([1, 256])
torch.Size([256, 120])


100%|██████████| 64/64 [00:00<00:00, 94.29it/s] 


In [215]:
output_fc- ref_fc

tensor([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
  